In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import random
import copy
import torch.nn as nn
import torch.nn.functional as F

from array import array




np.set_printoptions(precision=3)

In [2]:
class Car:
    def __init__(self, tyre="Intermediate"):
        self.default_tyre = tyre
        self.possible_tyres = ["Ultrasoft", "Soft", "Intermediate", "Fullwet"]
        self.pitstop_time = 23
        self.reset()


    def reset(self):
        self.change_tyre(self.default_tyre)


    def degrade(self, w, r):
        if self.tyre == "Ultrasoft":
            self.condition *= (1 - 0.0050*w - (2500-r)/90000)
        elif self.tyre == "Soft":
            self.condition *= (1 - 0.0051*w - (2500-r)/93000)
        elif self.tyre == "Intermediate":
            self.condition *= (1 - 0.0052*abs(0.5-w) - (2500-r)/95000)
        elif self.tyre == "Fullwet":
            self.condition *= (1 - 0.0053*(1-w) - (2500-r)/97000)


    def change_tyre(self, new_tyre):
        assert new_tyre in self.possible_tyres
        self.tyre = new_tyre
        self.condition = 1.00


    def get_velocity(self):
        if self.tyre == "Ultrasoft":
            vel = 80.7*(0.2 + 0.8*self.condition**1.5)
        elif self.tyre == "Soft":
            vel = 80.1*(0.2 + 0.8*self.condition**1.5)
        elif self.tyre == "Intermediate":
            vel = 79.5*(0.2 + 0.8*self.condition**1.5)
        elif self.tyre == "Fullwet":
            vel = 79.0*(0.2 + 0.8*self.condition**1.5)
        return vel


class Track:
    def __init__(self, car=Car()):
        # self.radius and self.cur_weather are defined in self.reset()
        self.total_laps = 162
        self.car = car
        self.possible_weather = ["Dry", "20% Wet", "40% Wet", "60% Wet", "80% Wet", "100% Wet"]
        self.wetness = {
            "Dry": 0.00, "20% Wet": 0.20, "40% Wet": 0.40, "60% Wet": 0.60, "80% Wet": 0.80, "100% Wet": 1.00
        }
        self.p_transition = {
            "Dry": {
                "Dry": 0.987, "20% Wet": 0.013, "40% Wet": 0.000, "60% Wet": 0.000, "80% Wet": 0.000, "100% Wet": 0.000
            },
            "20% Wet": {
                "Dry": 0.012, "20% Wet": 0.975, "40% Wet": 0.013, "60% Wet": 0.000, "80% Wet": 0.000, "100% Wet": 0.000
            },
            "40% Wet": {
                "Dry": 0.000, "20% Wet": 0.012, "40% Wet": 0.975, "60% Wet": 0.013, "80% Wet": 0.000, "100% Wet": 0.000
            },
            "60% Wet": {
                "Dry": 0.000, "20% Wet": 0.000, "40% Wet": 0.012, "60% Wet": 0.975, "80% Wet": 0.013, "100% Wet": 0.000
            },
            "80% Wet": {
                "Dry": 0.000, "20% Wet": 0.000, "40% Wet": 0.000, "60% Wet": 0.012, "80% Wet": 0.975, "100% Wet": 0.013
            },
            "100% Wet": {
                "Dry": 0.000, "20% Wet": 0.000, "40% Wet": 0.000, "60% Wet": 0.000, "80% Wet": 0.012, "100% Wet": 0.988
            }
        }
        self.num_actions = 5
        self.actions = [a for a in range(self.num_actions)]
        self.reset()


    def reset(self):
        self.radius = np.random.randint(600,1201)
        self.cur_weather = np.random.choice(self.possible_weather)
        self.is_done = False
        self.pitstop = False
        self.laps_cleared = 0
        self.car.reset()
        return self._get_state()


    def _get_state(self):
        return [self.car.tyre, self.car.condition, self.cur_weather, self.radius, self.laps_cleared]


    def transition(self, action=0):
        """
        Args:
            action (int):
                0. Make a pitstop and fit new ‘Ultrasoft’ tyres
                1. Make a pitstop and fit new ‘Soft’ tyres
                2. Make a pitstop and fit new ‘Intermediate’ tyres
                3. Make a pitstop and fit new ‘Fullwet’ tyres
                4. Continue the next lap without changing tyres
        """
        ## Pitstop time will be added on the first eight of the subsequent lap
        time_taken = 0
        if self.laps_cleared == int(self.laps_cleared):
            if self.pitstop:
                self.car.change_tyre(self.committed_tyre)
                time_taken += self.car.pitstop_time
                self.pitstop = False

        ## The environment is coded such that only an action taken at the start of the three-quarters mark of each lap matters
        if self.laps_cleared - int(self.laps_cleared) == 0.75:
            #print(self.laps_cleared)
            if action < 4:
                #print(action)
                self.pitstop = True
                self.committed_tyre = self.car.possible_tyres[action]
            else:
                self.pitstop = False

        self.cur_weather = np.random.choice(
            self.possible_weather, p=list(self.p_transition[self.cur_weather].values())
        )
        # we assume that degration happens only after a car has travelled the one-eighth lap
        velocity = self.car.get_velocity()
        time_taken += (2*np.pi*self.radius/8) / velocity
        reward = 0 - time_taken
        self.car.degrade(
            w=self.wetness[self.cur_weather], r=self.radius
        )
        self.laps_cleared += 0.125

        ##### Intro crash

       # if self.cur_weather != "Dry":
            ### calculate probability of crash

            # p_crash = _______________
        #    if np.random.rand() < p_crash:
         #       self.is_done = True

          #  else:




        if self.laps_cleared == self.total_laps:
            self.is_done = True

        next_state = self._get_state()
        return reward, next_state, self.is_done, velocity

In [3]:
def moving_average(values, window_size):
    """Calculate moving average with a given window size."""
    cumsum = np.cumsum(values)
    cumsum[window_size:] = cumsum[window_size:] - cumsum[:-window_size]
    return cumsum[window_size - 1:] / window_size

In [4]:
class Replay_buffer():
    '''
    Code based on:
    https://github.com/openai/baselines/blob/master/baselines/deepq/replay_buffer.py
    Expects tuples of (state, next_state, action, reward, done)
    '''
    def __init__(self, max_size):
        """Create Replay buffer.
        Parameters
        ----------
        size: int
            Max number of transitions to store in the buffer. When the buffer
            overflows the old memories are dropped.
        """
        self.storage = []
        self.max_size = max_size
        self.ptr = 0

    def push(self, data):
        if len(self.storage) == self.max_size:
            self.storage[int(self.ptr)] = data
            self.ptr = (self.ptr + 1) % self.max_size
        else:
            self.storage.append(data)

    def sample(self, batch_size):
        """Sample a batch of experiences.
        Parameters
        ----------
        batch_size: int
            How many transitions to sample.
        Returns
        -------
        state: np.array
            batch of state or observations
        action: np.array
            batch of actions executed given a state
        reward: np.array
            rewards received as results of executing action
        next_state: np.array
            next state next state or observations seen after executing action
        done: np.array
            done[i] = 1 if executing ation[i] resulted in
            the end of an episode and 0 otherwise.
        """
        ind = np.random.randint(0, len(self.storage), size=batch_size)
        state, next_state, action, reward, is_done = [], [], [], [], []

        #replay_buffer.push((state, next_state, action, reward, is_done))


        for i in ind:
            st, n_st, act, rew, dn = self.storage[i]
            state.append(st)
            next_state.append(n_st)
            action.append(np.array(act, copy=False))
            reward.append(np.array(rew, copy=False))
            is_done.append(np.array(dn, copy=False))

        return state, next_state, np.array(action), np.array(reward).reshape(-1, 1), np.array(is_done).reshape(-1, 1)


In [5]:
class Actor(nn.Module):
    """
    The Actor model takes in a state observation as input and
    outputs an action, which is a continuous value.

    It consists of four fully connected linear layers with ReLU activation functions and
    a final output layer selects one single optimized action for the state
    """
    def __init__(self, n_states, action_dim, hidden1):
        super(Actor, self).__init__()
        self.epsilon = 0.1
        self.net = nn.Sequential(
            nn.Linear(n_states, hidden1),
            nn.ReLU(),
            nn.Linear(hidden1, hidden1),
            nn.ReLU(),
            nn.Linear(hidden1, hidden1),
            nn.ReLU(),
            nn.Linear(hidden1, 5)
        )

    def forward(self, state):
        output = self.net(state)
        
        softmax_output = torch.softmax(output, dim=1)
        
        #print(softmax_output)
        
        if np.random.uniform(0,1) < self.epsilon:
            choice = torch.randint(low=0, high=5,size=())
        else:
            choice = torch.argmax(softmax_output)
            
        #print("choice is: ",choice," and softmax is: ",torch.argmax(softmax_output))
        return choice

class Critic(nn.Module):
    """
    The Critic model takes in both a state observation and an action as input and
    outputs a Q-value, which estimates the expected total reward for the current state-action pair.

    It consists of four linear layers with ReLU activation functions,
    State and action inputs are concatenated before being fed into the first linear layer.

    The output layer has a single output, representing the Q-value
    """

    def __init__(self, n_states, action_dim, hidden2):
        super(Critic, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n_states + action_dim, hidden2),
            nn.ReLU(),
            nn.Linear(hidden2, hidden2),
            nn.ReLU(),
            nn.Linear(hidden2, hidden2),
            nn.ReLU(),
            nn.Linear(hidden2, action_dim)
        )

        #print(self.net)



    def forward(self, state, action):
        return self.net(torch.cat((state.reshape(1,5), action.reshape(1,1)),1))

In [6]:
class OU_Noise(object):
    """Ornstein-Uhlenbeck process.
    code from :
    https://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
    The OU_Noise class has four attributes

        size: the size of the noise vector to be generated
        mu: the mean of the noise, set to 0 by default
        theta: the rate of mean reversion, controlling how quickly the noise returns to the mean
        sigma: the volatility of the noise, controlling the magnitude of fluctuations
    """
    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample.
        This method uses the current state of the noise and generates the next sample
        """
        dx = self.theta * (self.mu - self.state) + self.sigma * np.array([np.random.normal() for _ in range(len(self.state))])
        self.state += dx
        return self.state

In [7]:
#Set Hyperparameters
# Hyperparameters adapted for performance from
#https://ai.stackexchange.com/questions/22945/ddpg-doesnt-converge-for-mountaincarcontinuous-v0-gym-environment
capacity=10000
batch_size=64
update_iteration=100
tau=0.001 # tau for soft updating
gamma=0.99 # discount factor
directory = './'
hidden1=64 # hidden layer for actor
hidden2=64 #hiiden laye for critic

class DDPG(object):
    def __init__(self, state_dim, action_dim):
        """
        Initializes the DDPG agent.
        Takes three arguments:
               state_dim which is the dimensionality of the state space,
               action_dim which is the dimensionality of the action space, and
               max_action which is the maximum value an action can take.

        Creates a replay buffer, an actor-critic  networks and their corresponding target networks.
        It also initializes the optimizer for both actor and critic networks alog with
        counters to track the number of training iterations.
        """
        self.replay_buffer = Replay_buffer(max_size=capacity)

        self.actor = Actor(state_dim, action_dim, hidden1).to(device)
        self.actor_target = Actor(state_dim, action_dim,  hidden1).to(device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=3e-3)

        self.critic = Critic(state_dim, action_dim,  hidden2).to(device)
        self.critic_target = Critic(state_dim, action_dim,  hidden2).to(device)
        self.critic_target.load_state_dict(self.critic.state_dict())

        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=2e-2)
        # learning rate




        self.num_critic_update_iteration = 0
        self.num_actor_update_iteration = 0
        self.num_training = 0

    def discretize_state(self, state):
        tyre, condition, cur_weather, radius, laps_cleared = state

        tyre_map = {"Ultrasoft":1,
                     "Soft":2,
                     "Intermediate":3,
                     "Fullwet":4}

        cur_weather_map = {"Dry":1,
                    "20% Wet":2,
                    "40% Wet":3,
                    "60% Wet":4,
                    "80% Wet":5,
                    "100% Wet":6}

        #print(np.array([tyre_map[tyre], condition, cur_weather_map[cur_weather], radius, laps_cleared]))

        return (np.array([tyre_map[tyre], condition, cur_weather_map[cur_weather], radius, laps_cleared]))

    def select_action(self, state):
        #print(state)
        """
        takes the current state as input and returns an action to take in that state.
        It uses the actor network to map the state to an action.
        """
        state = torch.FloatTensor(self.discretize_state(state).reshape(1, -1)).to(device)
        #print("item:  ",round(self.actor(state).cpu().data.numpy().flatten()[0]))
        return round(self.actor(state).cpu().data.numpy().flatten()[0])


    def update(self):
        """
        updates the actor and critic networks using a batch of samples from the replay buffer.
        For each sample in the batch, it computes the target Q value using the target critic network and the target actor network.
        It then computes the current Q value
        using the critic network and the action taken by the actor network.

        It computes the critic loss as the mean squared error between the target Q value and the current Q value, and
        updates the critic network using gradient descent.

        It then computes the actor loss as the negative mean Q value using the critic network and the actor network, and
        updates the actor network using gradient ascent.

        Finally, it updates the target networks using
        soft updates, where a small fraction of the actor and critic network weights are transferred to their target counterparts.
        This process is repeated for a fixed number of iterations.
        """

        for it in range(update_iteration):
            # For each Sample in replay buffer batch

            st, n_st, act, rew, is_d = self.replay_buffer.sample(batch_size)

            for i in range(batch_size):
                
                #print("test: ",i," ",act[i])
                state = torch.FloatTensor(self.discretize_state(st[i])).to(device)

                action = torch.tensor(act[i]).to(device)
                #print(i," ",act[i])
                #print(action)
                next_state = torch.FloatTensor(self.discretize_state(n_st[i])).to(device)
                done = torch.FloatTensor(is_d[i]).to(device)
                reward = torch.FloatTensor(rew[i]).to(device)

                # Compute the target Q value
                #print("HERE ",(torch.unsqueeze(next_state, 0)))
                target_Q = self.critic_target(next_state, self.actor_target(torch.unsqueeze(next_state, 0)))

                target_Q = reward + (done * gamma * target_Q).detach()

                # Get current Q estimate
                #print('ok 221')
                #print(self.actor_target(next_state))
                #print(action)
                current_Q = self.critic(state, action)
                #print('ok 222')
                # Compute critic loss
                critic_loss = F.mse_loss(current_Q, target_Q)

                # Optimize the critic
                self.critic_optimizer.zero_grad()
                critic_loss.backward()
                self.critic_optimizer.step()
                # Compute actor loss as the negative mean Q value using the critic network and the actor network
                actor_loss = -self.critic(state, self.actor(torch.unsqueeze(state, 0))).mean()

                  # Optimize the actor
                self.actor_optimizer.zero_grad()
                actor_loss.backward()
                self.actor_optimizer.step()


            """
            Update the frozen target models using
            soft updates, where
            tau,a small fraction of the actor and critic network weights are transferred to their target counterparts.
            """
            for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

            for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)


            self.num_actor_update_iteration += 1
            self.num_critic_update_iteration += 1

            #print("end of iter num: ",it)

    def save(self):
        """
        Saves the state dictionaries of the actor and critic networks to files
        """
        torch.save(self.actor.state_dict(), directory + 'actor.pth')
        torch.save(self.critic.state_dict(), directory + 'critic.pth')

    def load(self):
        """
        Loads the state dictionaries of the actor and critic networks to files
        """
        self.actor.load_state_dict(torch.load(directory + 'actor.pth'))
        self.critic.load_state_dict(torch.load(directory + 'critic.pth')) #Train the agent on a Mountain car using DDPG

In [8]:
new_car = Car()
env = Track()
results = {}
max_episode=100
max_time_steps=5000
max_action=5
ep_r = 0
total_step = 0
rewards=[]
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [9]:
# Create a DDPG instance
state_dim = 5
action_dim = 1
agent = DDPG(state_dim, action_dim)

# Train the agent for max_episodes


for i in tqdm(range(max_episode)):
    G = 0
    is_done = False
    state = env.reset()

    while not is_done:
        #print(state)
        action = agent.select_action(state)
        # Add Gaussian noise to actions for exploration
        #action = (action + np.random.normal(0, 1, size=action_dim)).clip(1, max_action)
        #action += ou_noise.sample()

        reward, next_state, is_done, _ = env.transition(action)

        #reward, next_state, self.is_done, velocity

        G += reward

        agent.replay_buffer.push((state, next_state, action, reward, is_done))
        #print(state)
        state = next_state


    rewards.append(G)
    #total_step += step+1
    #print("Episode: \t{}  Total Reward: \t{:0.2f}".format( i, total_reward))
    agent.update()
    if i % 10 == 0:
        agent.save()
    #print("End of EP ",i)
#env.close()

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [35:12<00:00, 21.12s/it]


In [10]:
rewards

[-18925.28519367498,
 -15432.219126366757,
 -18061.681266956333,
 -14254.401022549195,
 -20024.968844123243,
 -18378.48871098118,
 -12938.826464198337,
 -17823.383201253546,
 -12183.565283357791,
 -13879.950006925996,
 -18911.604609284823,
 -17617.180527860706,
 -13905.861492924785,
 -13190.990752492851,
 -19268.42566810492,
 -12789.664687421979,
 -14428.683931419246,
 -17304.198020997712,
 -12682.681358497299,
 -14242.11217978202,
 -16045.817460669059,
 -14646.000485237837,
 -18182.097132590472,
 -17871.410078784927,
 -15649.647522448959,
 -19642.31599662163,
 -16636.492114480767,
 -15829.74430368892,
 -16996.684092184372,
 -12986.067168163145,
 -16389.716281513272,
 -12262.314751264483,
 -19779.69436014797,
 -19422.542954490193,
 -19075.847297252847,
 -15015.801554636022,
 -13432.638640273066,
 -18550.992215934966,
 -18818.727764982763,
 -19454.487485171525,
 -14205.855217018054,
 -16813.89595055127,
 -18030.23344253209,
 -12939.462578175095,
 -12504.474571274235,
 -18920.95933256699

In [12]:
test_iteration=100

for i in (range(test_iteration)):
    G = 0
    is_done = False
    state = env.reset()

    while not is_done:
        action = agent.select_action(state)
        reward, next_state, is_done, _ = env.transition(action)
        G += reward
        if is_done:
            print("Episode \t{}, the episode reward is \t{:0.2f}".format(i, G))
            print("radius is :",state[3])
            G = 0
            break
        state = next_state

Episode 	0, the episode reward is 	-14807.30
radius is : 799
Episode 	1, the episode reward is 	-18012.91
radius is : 1034
Episode 	2, the episode reward is 	-12440.03
radius is : 625
Episode 	3, the episode reward is 	-16390.67
radius is : 921
Episode 	4, the episode reward is 	-13516.56
radius is : 702
Episode 	5, the episode reward is 	-19865.36
radius is : 1179
Episode 	6, the episode reward is 	-20001.17
radius is : 1196
Episode 	7, the episode reward is 	-12047.91
radius is : 600
Episode 	8, the episode reward is 	-15406.79
radius is : 844
Episode 	9, the episode reward is 	-17714.05
radius is : 1023
Episode 	10, the episode reward is 	-17583.21
radius is : 1004
Episode 	11, the episode reward is 	-16803.63
radius is : 954
Episode 	12, the episode reward is 	-13913.04
radius is : 732
Episode 	13, the episode reward is 	-19626.58
radius is : 1175
Episode 	14, the episode reward is 	-15351.94
radius is : 840
Episode 	15, the episode reward is 	-15119.69
radius is : 832
Episode 	16,